# Data Science with Kaggle Decal 
## Spring 2017
## Day 6: Linear Regression

In [ ]:
import pandas as pd 
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression

## NBA Salary Data

In [ ]:
nba_sals = pd.read_csv("./nbasalary.csv", index_col = 0)
nba_sals = nba_sals.dropna(axis=0)
nba_sals.head()

In [ ]:
log_wage = nba_sals["lwage"]
wage = nba_sals["wage"]
points = nba_sals["points"]
exper = nba_sals["exper"]

## Simple Linear Regression

##### In this section, we will compare two SLR models and see which one performs better on a validation set. 
##### Model 1: Regressing wage on points scored
##### Model 2: Regressing wage on years of experience

In [ ]:
plt.figure(1)
plt.scatter(points,wage)
plt.title("Wage vs. Points")
plt.xlabel("Points")
plt.ylabel("Wage")
plt.show()
plt.close()

plt.figure(2)
plt.scatter(exper,wage)
plt.title("Wage vs. Experience")
plt.xlabel("Experience")
plt.ylabel("Wage")

plt.show()

In [ ]:
plt.figure(1)
myOLS_points = sm.OLS(wage,points).fit()
plt.plot(points, myOLS_points.predict(points))
plt.scatter(points, wage)
plt.title("Wage vs. Points")
plt.xlabel("Points")
plt.ylabel("Wage")
plt.show()
plt.close()

plt.figure(2)
myOLS_exper = sm.OLS(wage,exper).fit()
plt.plot(exper, myOLS_exper.predict(exper))
plt.scatter(exper, wage)
plt.title("Wage vs. Expereince")
plt.xlabel("Experience")
plt.ylabel("Wage")
plt.show()

## A little validation...

In [ ]:
wage_train = nba_sals["wage"][0:214]
wage_valid = nba_sals["wage"][214:]
points_train = nba_sals["points"][0:214]
points_valid = nba_sals["points"][214:]
exper_train = nba_sals["exper"][0:214]
exper_valid = nba_sals["exper"][214:]

#### Regression wage on points...

In [ ]:
myOLS = sm.OLS(wage_train,points_train).fit()
wage_hat = myOLS.predict(points_valid)
mse = 1/len(wage_valid)*np.dot((wage_valid - wage_hat),(wage_valid - wage_hat))
print("The MSE for the model wage~points is:", mse)

#### Regressing wage on experience...

In [ ]:
myOLS = sm.OLS(wage_train,exper_train).fit()
wage_hat = myOLS.predict(exper_valid)
mse = 1/len(wage_valid)*np.dot((wage_valid - wage_hat),(wage_valid - wage_hat))
print("The MSE for the model wage~experience is:", mse)

### Conclusion: points scored is a better predictor of wage than years of experience

## Multiple Linear Regression

### Wage vs. Experience & Points

In [ ]:
exp_dat = nba_sals[["exper","points"]]
exp_dat = sm.add_constant(exp_dat)
myMLR = sm.OLS(wage, exp_dat).fit()
myMLR.summary()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

x1, x2 = np.meshgrid(np.linspace(exp_dat.exper.min(), exp_dat.exper.max(), 100), 
                       np.linspace(exp_dat.points.min(), exp_dat.points.max(), 100))

x3 = myMLR.params[0] + myMLR.params[1] * x1 + myMLR.params[2] * x2

# create matplotlib 3d axes
fig = plt.figure(figsize=(12, 8))
my3D = Axes3D(fig, azim=-120, elev=20)

# plot hyperplane
surf = my3D.plot_surface(x1, x2, x3, cmap=plt.cm.RdBu_r, alpha=0.5, linewidth=0.5)

# plot data points
resid = wage - myMLR.predict(exp_dat)
my3D.scatter(exp_dat[resid >= 0].exper, exp_dat[resid >= 0].points, wage[resid >= 0], color='black', alpha=1.0, facecolor='white')
my3D.scatter(exp_dat[resid < 0].exper, exp_dat[resid < 0].points, wage[resid < 0], color='black', alpha=1.0)

# set axis labels
my3D.set_xlabel('experience')
my3D.set_ylabel('points')
my3D.set_zlabel('wage')
my3D.set_title('Regression Plane in 3D')

plt.show()

## Residual Plots

#### Residuals vs. fitted values

In [ ]:
wage_hat = myMLR.predict(exp_dat)
resids = wage - wage_hat

# Line of best fit should be Y = 0
residOLS = sm.OLS(resids,wage_hat).fit()

# Residuals vs. fitted falues
plt.plot(wage_hat, residOLS.predict(wage_hat))
plt.scatter(wage_hat, resids)
plt.ylabel("Residuals")
plt.xlabel("Fitted Values")
plt.title("Residuals vs. Fitted Values")
plt.plot(wage_hat, np.repeat(2,len(wage_hat)),color = "r")
plt.plot(wage_hat, np.repeat(-2,len(wage_hat)),color = "r")
plt.show()

#### Histogram of Residuals

In [ ]:
n, bins, patches = plt.hist(resids, bins = 20, normed= True, facecolor='green', alpha=0.5)

## pdf of a normal(0, std(resids))
y = mlab.normpdf( bins, np.mean(resids), np.std(resids))
l = plt.plot(bins, y, 'r', linewidth=2)
plt.title("Histogram of Residuals")
plt.show()

# Recursive Feature Elimination

#### From the API: "Feature ranking with recursive feature elimination and cross-validated selection of the best number of features."
#### "First, the estimator is trained on the initial set of features and weights are assigned to each one of them. Then, features whose absolute weights are the smallest are pruned from the current set features. That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached."
#### RFECV performs RFE with a cross-validation loop

In [ ]:
# Let's see what we have to work with...
nba_sals.columns

In [ ]:
# Let's pick a subset
nba_dat = nba_sals[["exper", "games","minutes","forward","center","points","guard","age"]]
nba_dat = sm.add_constant(nba_dat)

In [ ]:
myReg = LinearRegression()
myRFE = RFECV(myReg, step = 1, cv = 5)
myRFE = myRFE.fit(nba_dat[[1,2,3,4,5,6,7,8]], wage)

print(myRFE.support_)
print(myRFE.ranking_)

### Best model

In [ ]:
myMLR2 = sm.OLS(wage, nba_dat[[0,1,5,6,7]]).fit()
myMLR2.summary()

### Let's get the MSE of our best model...

In [ ]:
# Split into training and validation sets
nba_sals_train = nba_dat[0:214]
nba_sals_valid = nba_dat[214:]

wage_train = nba_sals["wage"][0:214]
wage_valid = nba_sals["wage"][214:]

nba_dat_train = nba_sals_train[[0,1,5,6,7]]
nba_dat_train = sm.add_constant(nba_dat_train)

nba_dat_valid = nba_sals_valid[[0,1,5,6,7]]
nba_dat_valid = sm.add_constant(nba_dat_valid)

In [ ]:
wage_hat = myMLR2.predict(nba_dat_valid)
mse = 1/len(wage_valid)*np.dot((wage_valid - wage_hat),(wage_valid - wage_hat))
print("The MSE of our best model is:", mse)